In [36]:
import pandas as pd

In [37]:
df_lego_scrap_brickeco = pd.read_csv('../04_Extra/APP/data/scraped_lego_data.csv')

In [38]:
df_lego_scrap_brickeco

,Number,SetName,Theme,Year,Pieces,Minifigs,RetailPriceUSD,CurrentValueNew,ForecastValueNew2Y,ForecastValueNew5Y,RollingGrowthLastYear,RollingGrowth12M,PriceType,PriceDate,PriceValue,Currency,URL,CurrentValueUsed
0,9840-1,Storage Solution 6 Pack,Education,2006,12.0,NaN,137.95,162.06,166.49,173.13,-2.14,-2.14,New,2025-02-04,162.06,USD,https://www.brickeconomy.com/api/v1/set/9840,NaN
1,9840-1,Storage Solution 6 Pack,Education,2006,12.0,NaN,137.95,162.06,166.49,173.13,-2.14,-2.14,New,2025-01-01,145.99,USD,https://www.brickeconomy.com/api/v1/set/9840,NaN
2,9840-1,Storage Solution 6 Pack,Education,2006,12.0,NaN,137.95,162.06,166.49,173.13,-2.14,-2.14,New,2024-12-02,153.26,USD,https://www.brickeconomy.com/api/v1/set/9840,NaN
3,9840-1,Storage Solution 6 Pack,Education,2006,12.0,NaN,137.95,162.06,166.49,173.13,-2.14,-2.14,New,2024-11-02,161.61,USD,https://www.brickeconomy.com/api/v1/set/9840,NaN
4,9840-1,Storage Solution 6 Pack,Education,2006,12.0,NaN,137.95,162.06,166.49,173.13,-2.14,-2.14,New,2024-10-14,164.29,USD,https://www.brickeconomy.com/api/v1/set/9840,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4544,6476267-1,Star Wars Celebration Europe 2023 Promotional ...,Star Wars,2023,1.0,NaN,NaN,22.93,26.06,32.73,-0.68,-0.68,New,2024-02-15,52.59,USD,https://www.brickeconomy.com/api/v1/set/6476267,NaN
4545,6476267-1,Star Wars Celebration Europe 2023 Promotional ...,Star Wars,2023,1.0,NaN,NaN,22.93,26.06,32.73,-0.68,-0.68,New,2024-02-01,51.24,USD,https://www.brickeconomy.com/api/v1/set/6476267,NaN
4546,6476267-1,Star Wars Celebration Europe 2023 Promotional ...,Star Wars,2023,1.0,NaN,NaN,22.93,26.06,32.73,-0.68,-0.68,New,2024-01-12,70.33,USD,https://www.brickeconomy.com/api/v1/set/6476267,NaN
4547,6476267-1,Star Wars Celebration Europe 2023 Promotional ...,Star Wars,2023,1.0,NaN,NaN,22.93,26.06,32.73,-0.68,-0.68,New,2024-01-07,63.66,USD,https://www.brickeconomy.com/api/v1/set/6476267,NaN


***LIMPIEZA Y NORMALIZACIÓN***

In [39]:
# Ordenamos por 'Number' y 'PriceDate' para garantizar el orden cronológico
df_lego_scrap_brickeco_sorted = df_lego_scrap_brickeco.sort_values(by=['Number', 'PriceDate'], ascending=[True, True])

# Asignamos un índice secuencial correcto para cada set sin saltos en los precios
df_lego_scrap_brickeco_sorted['PriceIndex'] = df_lego_scrap_brickeco_sorted.groupby('Number').cumcount()

# Hacemos un pivotable para reorganizar los precios correctamente
df_transformed = df_lego_scrap_brickeco_sorted.pivot(index=['Number', 'SetName', 'Theme', 'Year', 'Pieces', 'Minifigs', 
                                                             'RetailPriceUSD', 'CurrentValueNew', 'ForecastValueNew2Y', 
                                                             'ForecastValueNew5Y', 'RollingGrowthLastYear', 'RollingGrowth12M', 
                                                             'PriceType', 'Currency', 'URL', 'CurrentValueUsed'],
                                                      columns='PriceIndex', values='PriceValue').reset_index()

# Nos aseguramos que las columnas de precios estén correctamente numeradas y sin saltos en los datos
df_transformed.columns = [f'Price_{col+1}' if isinstance(col, int) else col for col in df_transformed.columns]

# Identificamos las columnas de precios
price_columns = [col for col in df_transformed.columns if col.startswith("Price_")]

# Reordeno los precios dentro de cada fila para eliminar espacios vacíos entre valores
def reorder_prices(row):
    prices = row[price_columns].dropna().values  # Obtenemos solo los valores no nulos
    new_row = [None] * len(price_columns)  # Creamos una nueva fila vacía con la misma cantidad de columnas
    new_row[:len(prices)] = prices  # Rellenamos desde el inicio sin dejar espacios
    return pd.Series(new_row, index=price_columns)

# Aplicamos la función a cada fila
df_transformed[price_columns] = df_transformed.apply(reorder_prices, axis=1)

# Eliminamos las columnas especificadas por no ser relevantes
columns_to_drop = ['Minifigs', 'RollingGrowthLastYear', 'RollingGrowth12M', 'CurrentValueUsed', 'Currency', 'URL', 'PriceType']

# Eliminamos las columnas de precios desde Price_13 hasta Price_24
price_columns_to_drop = [col for col in df_transformed.columns if col.startswith('Price_') and 13 <= int(col.split('_')[1]) <= 280]

# Combinamos todas las columnas a eliminar
columns_to_drop.extend(price_columns_to_drop)

# Ejecutamos el drop de las columnas seleccionadas
df_transformed_limpia = df_transformed.drop(columns=[col for col in columns_to_drop if col in df_transformed.columns])

# Reemplazamos valores nulos en las columnas 'Price_1' hasta 'Price_12' por 0
columns_to_fill = [f'Price_{i}' for i in range(1, 13)]
df_transformed_limpia[columns_to_fill] = df_transformed_limpia[columns_to_fill].fillna(0)

# Reemplazamos valores nulos en la columna 'Pieces' por 0
df_transformed_limpia['Pieces'] = df_transformed_limpia['Pieces'].fillna(0)

# Reemplazamos valores nulos en la columna 'RetailPriceUSD' por 0
df_transformed_limpia['RetailPriceUSD'] = df_transformed_limpia['RetailPriceUSD'].fillna(0)

# Reemplazamos valores 0 en 'CurrentValueNew' con el valor de 'RetailPriceUSD'
df_transformed_limpia.loc[df_transformed_limpia['CurrentValueNew'] == 0, 'CurrentValueNew'] = df_transformed_limpia['RetailPriceUSD']

# Eliminamos las filas con valores nulos
df_transformed_limpia = df_transformed_limpia.dropna()

In [40]:
df_transformed_limpia.isnull().sum()

Number                0
SetName               0
Theme                 0
Year                  0
Pieces                0
RetailPriceUSD        0
CurrentValueNew       0
ForecastValueNew2Y    0
ForecastValueNew5Y    0
Price_1               0
Price_2               0
Price_3               0
Price_4               0
Price_5               0
Price_6               0
Price_7               0
Price_8               0
Price_9               0
Price_10              0
Price_11              0
Price_12              0
dtype: int64

In [41]:
df_transformed_limpia 

,Number,SetName,Theme,Year,Pieces,RetailPriceUSD,CurrentValueNew,ForecastValueNew2Y,ForecastValueNew5Y,Price_1,...,Price_3,Price_4,Price_5,Price_6,Price_7,Price_8,Price_9,Price_10,Price_11,Price_12
0,10274-1,ECTO-1,Ghostbusters,2020,2352.0,239.99,300.00,348.16,419.26,229.99,...,222.08,219.96,218.61,217.41,215.44,210.05,210.04,279.34,286.12,300.00
1,10283-1,Space Shuttle Discovery,Icons,2021,2354.0,199.99,246.80,291.06,356.41,184.31,...,193.27,189.98,188.91,185.57,180.85,184.64,248.90,239.05,249.55,246.80
2,10303-1,Loop Coaster,Icons,2022,3756.0,399.99,425.29,484.00,570.67,400.62,...,380.28,386.48,360.60,399.85,395.45,396.50,399.96,438.12,419.46,425.29
3,10304-1,Chevrolet Camaro Z/28 1969,Icons,2022,1456.0,169.99,183.60,207.77,243.44,141.70,...,139.21,139.80,148.44,149.99,147.87,153.30,175.67,189.68,183.32,183.60
4,10306-1,Atari 2600,Icons,2022,2532.0,239.99,230.00,272.32,334.80,197.94,...,191.96,203.60,193.06,226.99,223.47,234.57,213.79,220.39,238.51,230.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,952310-1,Builder with Digger,City,2023,21.0,0.00,2.63,3.33,3.78,3.02,...,2.75,2.76,2.95,2.84,2.76,2.63,2.67,2.60,2.53,2.63
362,952311-1,Diver with Underwater Scooter,City,2023,27.0,0.00,3.18,3.70,4.46,3.80,...,3.78,3.76,3.88,3.82,3.85,3.53,3.37,3.50,3.28,3.18
363,952312-1,Snowmobile,City,2023,21.0,0.00,3.01,3.14,3.33,3.58,...,3.16,3.01,2.96,3.14,2.99,2.96,3.27,3.06,3.04,3.01
364,952401-1,Builder with Crane,City,2023,29.0,0.00,3.84,4.54,5.58,3.56,...,2.88,2.73,2.84,3.21,2.83,3.30,3.59,3.77,3.68,3.84


In [42]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# 🔹 Guardar las columnas de identificación antes de eliminarlas para el modelo
id_columns = ['Number', 'SetName', 'Theme', 'RetailPriceUSD', 'CurrentValueNew']
df_identification = df_transformed_limpia[id_columns]

# 🔹 Mantener las columnas de precios históricos Price_1 a Price_12
price_columns = [col for col in df_transformed_limpia.columns if col.startswith('Price_')]

# 🔹 Crear una copia del dataset sin las columnas de identificación pero con precios históricos
df_model = df_transformed_limpia.drop(columns=['Number', 'SetName', 'Theme'], errors='ignore')

# 🔹 Convertir variables categóricas en dummies si quedan (por si Theme es categórico)
df_model = pd.get_dummies(df_model, drop_first=True)

# 🔹 Definir variables de entrada (X) y salida (y)
X = df_model.drop(columns=['ForecastValueNew2Y', 'ForecastValueNew5Y'])  # Features (incluye precios históricos)
y_2y = df_model['ForecastValueNew2Y']  # Target a 2 años
y_5y = df_model['ForecastValueNew5Y']  # Target a 5 años

# 🔹 Separación en conjuntos de entrenamiento y prueba
X_train, X_test, y_train_2y, y_test_2y = train_test_split(X, y_2y, test_size=0.2, random_state=42)
X_train, X_test, y_train_5y, y_test_5y = train_test_split(X, y_5y, test_size=0.2, random_state=42)

# 🔹 Definir la búsqueda de hiperparámetros para XGBoost
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 🔹 Entrenar el modelo para 2 años con búsqueda de hiperparámetros
xgb_2y = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search_2y = GridSearchCV(xgb_2y, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search_2y.fit(X_train, y_train_2y)

# 🔹 Entrenar el modelo para 5 años con búsqueda de hiperparámetros
xgb_5y = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search_5y = GridSearchCV(xgb_5y, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search_5y.fit(X_train, y_train_5y)

# 🔹 Evaluación del mejor modelo en el conjunto de prueba
best_xgb_2y = grid_search_2y.best_estimator_
best_xgb_5y = grid_search_5y.best_estimator_

y_pred_2y = best_xgb_2y.predict(X_test)
y_pred_5y = best_xgb_5y.predict(X_test)

# 🔹 Métricas de evaluación
rmse_2y = np.sqrt(mean_squared_error(y_test_2y, y_pred_2y))
r2_2y = r2_score(y_test_2y, y_pred_2y)

rmse_5y = np.sqrt(mean_squared_error(y_test_5y, y_pred_5y))
r2_5y = r2_score(y_test_5y, y_pred_5y)

print("📊 **Resultados del modelo XGBoost**")
print(f"🔹 Predicción a 2 años -> RMSE: {rmse_2y:.2f}, R²: {r2_2y:.2f}")
print(f"🔹 Predicción a 5 años -> RMSE: {rmse_5y:.2f}, R²: {r2_5y:.2f}")

# 🔹 Mostrar los mejores hiperparámetros encontrados
print("\n🎯 Mejores hiperparámetros para XGBoost 2 años:", grid_search_2y.best_params_)
print("🎯 Mejores hiperparámetros para XGBoost 5 años:", grid_search_5y.best_params_)

# 🔹 Unimos las predicciones con la información de los sets
df_predictions = df_identification.copy()
df_predictions['PredictedValue2Y'] = best_xgb_2y.predict(X)
df_predictions['PredictedValue5Y'] = best_xgb_5y.predict(X)

df_predictions

📊 **Resultados del modelo XGBoost**
🔹 Predicción a 2 años -> RMSE: 8.69, R²: 0.99
🔹 Predicción a 5 años -> RMSE: 15.65, R²: 0.98

🎯 Mejores hiperparámetros para XGBoost 2 años: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
🎯 Mejores hiperparámetros para XGBoost 5 años: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.8}


,Number,SetName,Theme,RetailPriceUSD,CurrentValueNew,PredictedValue2Y,PredictedValue5Y
0,10274-1,ECTO-1,Ghostbusters,239.99,300.00,348.125977,418.969940
1,10283-1,Space Shuttle Discovery,Icons,199.99,246.80,290.876587,356.250275
2,10303-1,Loop Coaster,Icons,399.99,425.29,484.578156,570.797363
3,10304-1,Chevrolet Camaro Z/28 1969,Icons,169.99,183.60,202.167496,225.247864
4,10306-1,Atari 2600,Icons,239.99,230.00,272.239075,334.693451
...,...,...,...,...,...,...,...
361,952310-1,Builder with Digger,City,0.00,2.63,3.480175,3.854361
362,952311-1,Diver with Underwater Scooter,City,0.00,3.18,3.591519,4.135797
363,952312-1,Snowmobile,City,0.00,3.01,3.472350,3.733291
364,952401-1,Builder with Crane,City,0.00,3.84,4.412468,5.301325


In [43]:
import joblib

In [44]:
joblib.dump(best_xgb_2y, "../04_Extra/APP/models/xgb_2y.pkl")
joblib.dump(best_xgb_5y, "../04_Extra/APP/models/xgb_5y.pkl")

['../04_Extra/APP/models/xgb_5y.pkl']

In [45]:
df_predictions

,Number,SetName,Theme,RetailPriceUSD,CurrentValueNew,PredictedValue2Y,PredictedValue5Y
0,10274-1,ECTO-1,Ghostbusters,239.99,300.00,348.125977,418.969940
1,10283-1,Space Shuttle Discovery,Icons,199.99,246.80,290.876587,356.250275
2,10303-1,Loop Coaster,Icons,399.99,425.29,484.578156,570.797363
3,10304-1,Chevrolet Camaro Z/28 1969,Icons,169.99,183.60,202.167496,225.247864
4,10306-1,Atari 2600,Icons,239.99,230.00,272.239075,334.693451
...,...,...,...,...,...,...,...
361,952310-1,Builder with Digger,City,0.00,2.63,3.480175,3.854361
362,952311-1,Diver with Underwater Scooter,City,0.00,3.18,3.591519,4.135797
363,952312-1,Snowmobile,City,0.00,3.01,3.472350,3.733291
364,952401-1,Builder with Crane,City,0.00,3.84,4.412468,5.301325
